
docker run --name=deploy_trt1912 --rm -it --runtime nvidia -v ${PWD}:/py -w /py -p 20082:8888 nvcr.io/nvidia/tensorrt:19.12-py3 bash

pip3 install notebook

jupyter notebook --allow-root --ip 0.0.0.0

In [1]:
import sys
import os
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt
import numpy as np 


In [2]:
trt.__version__

'6.0.1.8'

In [3]:
!pip3 install onnx

You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import onnx
import onnx.helper as oh
from onnx import checker
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import sys
import os

AXIS = 1
SHAPE = [1, 3, 4, 5]
# SHAPE = [1, 1, 3, 4, 5]
ONNX_FILE = f"softmax_test{len(SHAPE)}axis{AXIS}.onnx"

# Generate ONNX model
def generate_model(io_shape=SHAPE, axis=AXIS):
    in_tensor = [
        oh.make_tensor_value_info("Input", onnx.TensorProto.FLOAT, io_shape)
    ]
    out_tensor = [
        oh.make_tensor_value_info("Output", onnx.TensorProto.FLOAT, io_shape)
    ]
    nodes = []
    nodes.append(oh.make_node("Softmax", axis=axis, \
        inputs=["Input"], outputs=["Output"]))
    graph = oh.make_graph(nodes, "Test Graph", in_tensor, out_tensor)
    checker.check_graph(graph)
    model = oh.make_model(graph, producer_name="MACNICA", \
        producer_version="0.1")
    checker.check_model(model)
    with open(ONNX_FILE, "wb") as f:
        f.write(model.SerializeToString())

generate_model(SHAPE, AXIS)
print(ONNX_FILE)

softmax_test4axis1.onnx


In [5]:
!trtexec --onnx=softmax_test4axis1.onnx --saveEngine=softmax_test4axis1_trt6.trt --explicitBatch --fp16 --verbose --dumpOutput

&&&& RUNNING TensorRT.trtexec # trtexec --onnx=softmax_test4axis1.onnx --saveEngine=softmax_test4axis1_trt6.trt --explicitBatch --fp16 --verbose --dumpOutput
[04/24/2021-13:32:55] [I] === Model Options ===
[04/24/2021-13:32:55] [I] Format: ONNX
[04/24/2021-13:32:55] [I] Model: softmax_test4axis1.onnx
[04/24/2021-13:32:55] [I] Output:
[04/24/2021-13:32:55] [I] === Build Options ===
[04/24/2021-13:32:55] [I] Max batch: explicit
[04/24/2021-13:32:55] [I] Workspace: 16 MB
[04/24/2021-13:32:55] [I] minTiming: 1
[04/24/2021-13:32:55] [I] avgTiming: 8
[04/24/2021-13:32:55] [I] Precision: FP16
[04/24/2021-13:32:55] [I] Calibration: 
[04/24/2021-13:32:55] [I] Safe mode: Disabled
[04/24/2021-13:32:55] [I] Save engine: softmax_test4axis1_trt6.trt
[04/24/2021-13:32:55] [I] Load engine: 
[04/24/2021-13:32:55] [I] Inputs format: fp32:CHW
[04/24/2021-13:32:55] [I] Outputs format: fp32:CHW
[04/24/2021-13:32:55] [I] Input build shapes: model
[04/24/2021-13:32:55] [I] === System Options ===
[04/24/2021-

In [6]:
TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE)
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print(f"Reading engine from file {engine_file_path}")
    with open(engine_file_path,"rb") as f , trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

def infer(engine):
    input_image = np.ones((1,3,4,5))
    with engine.create_execution_context() as context:
        context.set_binding_shape(engine.get_binding_index("input"),(1,3,4,5))
        bindings = []
        for binding in engine:
            binding_idx = engine.get_binding_index(binding)
            size = trt.volume(context.get_binding_shape(binding_idx))
            dtype = trt.nptype(engine.get_binding_dtype(binding))
            if engine.binding_is_input(binding):
                input_buffer = np.ascontiguousarray(input_image)
                input_memory = cuda.mem_alloc(input_image.nbytes)
                print("in",list(engine.get_binding_shape(binding)))
                bindings.append(int(input_memory))
            else:
                output_buffer = cuda.pagelocked_empty(size,dtype)
                output_memory = cuda.mem_alloc(output_buffer.nbytes)
                print("out",list(engine.get_binding_shape(binding)))
                bindings.append(int(output_memory))
        stream = cuda.Stream()
        
        cuda.memcpy_htod_async(input_memory,input_buffer,stream)
        context.execute_async_v2(bindings=bindings, stream_handle=stream.handle)
        cuda.memcpy_dtoh_async(output_buffer,output_memory,stream)
        
        stream.synchronize()
    print("engine",engine.max_batch_size)
    print("output_buffer",output_buffer.shape)
    print(output_buffer.reshape(SHAPE))

In [7]:
engine_file = "softmax_test4axis1_trt6.trt"
with load_engine(engine_file) as engine:
    infer(engine)

Reading engine from file softmax_test4axis1_trt6.trt
in [3, 4, 5]
out [3, 4, 5]
engine 1
output_buffer (60,)
[[[[0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]]

  [[0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]]

  [[0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]
   [0.33333334 0.33333334 0.33333334 0.33333334 0.33333334]]]]
